In [1]:
import sys, os 

import numpy as np
import pandas as pd

import scipy as sci
import xarray as xr

# Input

In [2]:
experiment_name = 'SODA'
region = 'CelticSea'

# Specification of biological parameters  
# Number of years in one life cycle of an individual 
generation = 2 
# The year when the input temeperature dataset starts 
# Should be one year less than starting year in the dataset
initial_year = 1999	        
# Number of years in the input temperature dataset
years = range(2000, 2005)  	 


# Define geographic boundaries (latitudes, longitudes)
# Default coordinates: North Atlantic coordinates
lat_coords = slice(47,52)
lon_coords = slice(-12,-1)

# Define depth_levels for your growth model output files
depth_levels = slice(0, 600)   # 0-600 meters according to Atlantic cod distribution

# Define latitudes that you will use to save your weight-at-age data to netcdf
lat = np.array([47.25, 47.75, 48.25, 48.75, 49.25, 49.75, 50.25, 50.75, 51.25, 51.75], dtype='f')
lon = np.array([
     -11.75, -11.25, -10.75, -10.25, -9.75, -9.25, -8.75, -8.25, -7.75,
    -7.25, -6.75, -6.25, -5.75, -5.25, -4.75, -4.25, -3.75, -3.25, -2.75, -2.25, -1.75,
    -1.25
], dtype='f')

# Define depth that you will use to save your weight-at-age data to netcdf
depths = np.array([
     -0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 115.,
     135., 160., 190., 230., 280., 340., 410., 490., 580.
], dtype='f')



# Constants

In [3]:
""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

# Functions

In [4]:
""" Definitions of kinetic functions """

def equation2(input_temp):
    T0 = 273.15                                  # O°C in K
    temperature_kelvin = input_temp + T0         
               
    # Calculate a
    a_numerator = A_R*np.exp(THETA_A/T_R - THETA_A/temperature_kelvin) 
    a_denominator = 1 + np.exp(THETA_H/T_H - THETA_H/temperature_kelvin)
    a = a_numerator/a_denominator
    return (a)


def equation3(input_temp):
    """ Arrhenius equation """
    T0 = 273.15                                # 0°C in K
    temperature_kelvin = input_temp + T0 

    # Calculate b
    b = B_R*np.exp(THETA_B/T_R - THETA_B/temperature_kelvin)
    return(b)

# Setup

In [8]:
initial_year = 1999	

lat_coords = lat_coords
lon_coords = lon_coords

depth_levels = depth_levels 

dt = 1

# coords for output
lat, lon = lat, lon 

depths = depths

generation = generation
# Should be one year less than starting year in the dataset
initial_year = initial_year
# Number of years in the input temperature dataset
years = years
# Define dimensionality of coords
N_depths = len(depths)
N_lat = len(lat)
N_lon = len(lon)

In [84]:
N_lat,N_lon

(10, 22)

In [34]:
input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})
input_files['thetao']

<xarray.DataArray 'thetao' (time: 60, depth_coord: 21, latitude: 10, longitude: 22)>
dask.array<concatenate, shape=(60, 21, 10, 22), dtype=float32, chunksize=(12, 21, 10, 22), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2004-12-31
  * longitude    (longitude) float64 -11.75 -11.25 -10.75 ... -2.25 -1.75 -1.25
  * latitude     (latitude) float64 47.25 47.75 48.25 ... 50.75 51.25 51.75
  * depth_coord  (depth_coord) float32 0.0 10.0 20.0 30.0 ... 410.0 490.0 580.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [90]:
input_files['thetao'][0][0][0][1].values.ravel()

array([nan])

In [117]:
m = input_files['thetao'].transpose('latitude','longitude', 'time', 'depth_coord').fillna(-999); m

<xarray.DataArray 'thetao' (latitude: 10, longitude: 22, time: 60, depth_coord: 21)>
dask.array<where, shape=(10, 22, 60, 21), dtype=float32, chunksize=(10, 22, 12, 21), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2004-12-31
  * longitude    (longitude) float64 -11.75 -11.25 -10.75 ... -2.25 -1.75 -1.25
  * latitude     (latitude) float64 47.25 47.75 48.25 ... 50.75 51.25 51.75
  * depth_coord  (depth_coord) float32 0.0 10.0 20.0 30.0 ... 410.0 490.0 580.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [118]:
m.values.ravel()

array([-999., -999., -999., ..., -999., -999., -999.], dtype=float32)

In [119]:
n=273

def doit():
    i = 0
    for lat in range(len(m.latitude)):
        for lon in range(len(m.longitude)):
            for d in range(len(m.depth_coord)):
                if i == n:
                    print(lat,lon,d)
                    return
                i+=1
doit()

0 13 0


In [121]:
m[0][13][0][0].values.ravel()[0]

12.1378145

In [124]:

len_lat, len_lon = len(m.latitude), len(m.longitude)
len_ts, len_depth = len(m.time), len(m.depth_coord)

output = np.arange(len_lat*len_lon*len_depth * len_ts).reshape(len_lat, len_lon, len_depth, len_ts).astype('float32')
m = input_files['thetao'].transpose('latitude','longitude', 'depth_coord', 'time').fillna(-999)

for lat in range(len_lat):
    for lon in range(len_lon):
        for depth in range(len_depth):
            output[lat,lon,depth] = m[lat,lon,depth,:].values.ravel()[0]

output = output.reshape(len_lat * len_lon * len_depth, len_ts)
# mask = output < -999; mask
# output[mask] = -999
np.savetxt('data_2000-2004.csv', output, delimiter=',', fmt='%1.7f')

# Compute

In [14]:
initial_year = 1999	

lat_coords = lat_coords
lon_coords = lon_coords

depth_levels = depth_levels 

dt = 1

# coords for output
lat, lon = lat, lon 

depths = depths

generation = generation
# Should be one year less than starting year in the dataset
initial_year = initial_year
# Number of years in the input temperature dataset
years = years
# Define dimensionality of coords
N_depths = len(depths)
N_lat = len(lat)
N_lon = len(lon)

input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})

for each_year in np.array(years).astype('int'):
    # Selecting the inital year (birth of cod)         
    initial_year = initial_year + 1
    print('INITIAL YEAR: ', initial_year)
    
    # Identifiyng the last year (death of cod)
    last_year = initial_year + generation         
    print('LAST YEAR: ', last_year)
    
    # Beginning of life cycle
    age = 0                                       
    
    weight = np.ones(shape=(N_depths, N_lat*N_lon), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
    for every_year in range(initial_year, last_year):
        if last_year > years[-1]: print('!'); break                                       
        else:
            my_temp = input_files.thetao.sel(
              time=f'{every_year}', 
              depth_coord = depth_levels, latitude = lat_coords, 
              longitude = lon_coords
            )            
            print(my_temp)
            break
    break
            
            # print('WORK ON YEAR: ', every_year)
            # # Partly vectorize 4D temperature fields 
            # # to accelerate the computations
            # temp_input_3d = my_temp.values.reshape(12, N_depths, N_lat*N_lon)             
               
            # # Set NaN values                         
          
            # temp_input_3d[np.where(temp_input_3d[:,:,:] <= -998)] = np.nan        
            # # Initialize necessary variable fields
            # a = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
            # b = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

            # if every_year == initial_year: mm0 = 2           
            # else: mm0 = 0
                
            # for mon in np.arange(0,12):
            #   for dd in np.arange(0,30):
            #     for ilev in np.arange(0, N_depths):  
            #       a[ilev, :] = equation2(temp_input_3d[mon, ilev, :])
            #       b[ilev, :] = equation3(temp_input_3d[mon, ilev, :]) * (-1.)
            #       growth_rates[ilev, :] = 0.01 * ( a[ilev, :] * weight[ilev, :]** b[ilev, :] - C_AVG )  
            #       growth_rates[ilev, :] = np.where(growth_rates[ilev, :] < 0, 0, growth_rates[ilev, :])
            #       weight[ilev, :] = weight[ilev, :] * (1. + dt  * growth_rates[ilev, :])
                                                                
            # new_year = int(every_year) + 1
            
            # # Reshape data to original shape
            # a_3d = a.reshape(N_depths, N_lat, N_lon)
            # b_3d = b.reshape(N_depths, N_lat, N_lon)
            # growth_rates_3d = growth_rates.reshape(N_depths, N_lat, N_lon)
            
            # # 3D field with asymptotic weight
            # weight_3d = 0.001 * weight.reshape((N_depths, N_lat, N_lon)) 
            
            # # Calculate maximum asymptotic weight at a given location 
            # # ("W*" in Butzin and Pörtner (2016)) 
            # weight_max = np.nanmax(weight_3d, axis = 0)
            





            # Create output directories if they do not exist
            
            # OUT = f'{output_dir}{folder_name}/init_{initial_year}_transient/{experiment_name}' 
            # output_a_3d = f'{OUT}/a_3d/'
            # output_b_3d = f'{OUT}/b_3d/'
            # output_growth_rates_3d = f'{OUT}/growth_rates/'
            # output_weight_3d = f'{OUT}/weight_3d/'
            # output_weight_max = f'{OUT}/weight_max/'
            
            # for F in [
            #     output_a_3d, output_b_3d, output_growth_rates_3d, 
            #     output_weight_3d, output_weight_max
            #     ]: 
            #     if not os.path.exists(F): os.makedirs(F)
                
            # print(OUT)
            # # Save netcdf
            # save_netcdf(a_3d, 'a_3d', output_a_3d, new_year, age)   
            # save_netcdf(b_3d, 'b_3d', output_b_3d, new_year, age)
            # save_netcdf(growth_rates_3d, 'growth_rates_3d', output_growth_rates_3d, new_year, age)
            # save_netcdf(weight_3d, 'weight_3d', output_weight_3d, new_year, age)
            # # Save weight_max           
            # save_netcdf(
            #     weight_max, 'weight_max', output_weight_max, new_year, 
            #     age, dimension_labels = ('latitude', 'longitude')
            # )

INITIAL YEAR:  2000
LAST YEAR:  2002
<xarray.DataArray 'thetao' (time: 12, depth_coord: 21, latitude: 10, longitude: 22)>
dask.array<getitem, shape=(12, 21, 10, 22), dtype=float32, chunksize=(12, 21, 10, 22), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2000-12-31
  * longitude    (longitude) float64 -11.75 -11.25 -10.75 ... -2.25 -1.75 -1.25
  * latitude     (latitude) float64 47.25 47.75 48.25 ... 50.75 51.25 51.75
  * depth_coord  (depth_coord) float32 0.0 10.0 20.0 30.0 ... 410.0 490.0 580.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt
